<a href="https://colab.research.google.com/github/DenShum/Statistics-in-Python/blob/main/%D0%94%D0%971_stat_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание к лекции "Базовые понятия статистики"

## Обязательная часть

Будем осуществлять работу с непростым [набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики. 

### Задание 1. Базовое изучение

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names) и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

### Задание 2. Работа с выбросами

В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

### Задание 3. Работа с пропусками

Рассчитать количество выбросов для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

## Дополнительная часть (необязательная)

Выполнить задания 1-3 для всего набора данных.

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам
Прежде чем задать вопрос, попробуйте найти ответ в интернете. Навык самостоятельного поиска информации — один из важнейших. Каждый практикующий специалист любого уровня делает это ежедневно.

Сформулируйте вопрос по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности прикрепите к вопросу скриншоты либо ссылки на код. Не выкладывайте все решение, оставляйте только проблемный и воспроизводимый участок кода.

***Читаем датафрейм, заменяем "?" на NaN, переименовываем заголовки и, фильтруя, оставляем в дата фрейме horse_data требуемые по условию задания заголовки***


In [ ]:
import pandas as  pd
df = pd.read_csv('horse_data.csv',na_values='?')
df.columns = ['surgery?','Age','3','rectal_temp','pulse','respiratory_rate','temp_of_extremities','8','9','10','pain','12','13','14','15','16','17','18','19','20','21','22','outcome','24','25','26','27','28']
horse_data = df[['surgery?','Age','rectal_temp','pulse','respiratory_rate','temp_of_extremities','pain','outcome']]
horse_data.head()

,surgery?,Age,rectal_temp,pulse,respiratory_rate,temp_of_extremities,pain,outcome
0,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
1,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
2,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
3,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
4,2.0,1,NaN,NaN,NaN,2.0,2.0,1.0


Выводим инфомацию по дата фрейму. Меняем типы данных на числовые - видим, что по некоторым колонкам есть пропуски (null). Выводим основные статистические показатели - видим, что есть выбросы:
- по Age -"9"(по описанию дата фрейма значения могут быть только 1 - взрослая лошадь и 2 - молодые(<6 месяцев)


  

In [ ]:
horse_data.info()
horse_data = horse_data.apply(pd.to_numeric,errors='coerce')
# horse_data.describe(include='all')
horse_data.describe().round()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   surgery?             298 non-null    float64
 1   Age                  299 non-null    int64  
 2   rectal_temp          239 non-null    float64
 3   pulse                275 non-null    float64
 4   respiratory_rate     241 non-null    float64
 5   temp_of_extremities  243 non-null    float64
 6   pain                 244 non-null    float64
 7   outcome              298 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.8 KB


,surgery?,Age,rectal_temp,pulse,respiratory_rate,temp_of_extremities,pain,outcome
count,298.0,299.0,239.0,275.0,241.0,243.0,244.0,298.0
mean,1.0,2.0,38.0,72.0,30.0,2.0,3.0,2.0
std,0.0,2.0,1.0,29.0,18.0,1.0,1.0,1.0
min,1.0,1.0,35.0,30.0,8.0,1.0,1.0,1.0
25%,1.0,1.0,38.0,48.0,18.0,1.0,2.0,1.0
50%,1.0,1.0,38.0,64.0,24.0,3.0,3.0,1.0
75%,2.0,1.0,38.0,88.0,36.0,3.0,4.0,2.0
max,2.0,9.0,41.0,184.0,96.0,4.0,5.0,3.0


***Удаляем выбросы (7.7 % от общего количества данных) по Age  по формуле межквартильного размаха***

In [ ]:
q1 = horse_data['Age'].quantile(0.25)
q3 = horse_data['Age'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_horse_data = horse_data[horse_data['Age'].between(lower_bound, upper_bound, inclusive=True)]
# pd.concat([horse_data, remove_horse_data]).drop_duplicates(keep=False) #выбросы по возрасту
# remove_horse_data.describe().round() #проверяем данные после очистки от выбросов по возрасту
remove_horse_data.info() # data frame очищенный от выбросов по возрасту

<class 'pandas.core.frame.DataFrame'>
Int64Index: 275 entries, 0 to 298
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   surgery?             274 non-null    float64
 1   Age                  275 non-null    int64  
 2   rectal_temp          219 non-null    float64
 3   pulse                253 non-null    float64
 4   respiratory_rate     221 non-null    float64
 5   temp_of_extremities  227 non-null    float64
 6   pain                 227 non-null    float64
 7   outcome              274 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 19.3 KB


In [ ]:
(remove_horse_data.isna().mean()*100).round(2) # % пропусков по колонкам

surgery?                0.36
Age                     0.00
rectal_temp            20.36
pulse                   8.00
respiratory_rate       19.64
temp_of_extremities    17.45
pain                   17.45
outcome                 0.36
dtype: float64

In [ ]:
remove_horse_data.describe().round() #статистические показатели при наличии пропусков

,surgery?,Age,rectal_temp,pulse,respiratory_rate,temp_of_extremities,pain,outcome
count,274.0,275.0,219.0,253.0,221.0,227.0,227.0,274.0
mean,1.0,1.0,38.0,67.0,28.0,2.0,3.0,2.0
std,0.0,0.0,1.0,24.0,15.0,1.0,1.0,1.0
min,1.0,1.0,35.0,30.0,8.0,1.0,1.0,1.0
25%,1.0,1.0,38.0,48.0,18.0,1.0,2.0,1.0
50%,1.0,1.0,38.0,60.0,24.0,3.0,3.0,1.0
75%,2.0,1.0,38.0,84.0,36.0,3.0,4.0,2.0
max,2.0,1.0,41.0,140.0,96.0,4.0,5.0,3.0


***В категориальных столбцах: 'surgery?', 'outcome', 'temp_of_extremities', 'pain' - пропуски заменяем модальным значением***

In [ ]:
remove_horse_data['surgery?'].fillna(remove_horse_data['surgery?'].mode()[0],inplace=True) # замена пропусков в surgery? - модой
# print(remove_horse_data['surgery?'].isna().sum()) #проверяем наличие NaN в колонке surgery?
# remove_horse_data['surgery?'].describe().round() # статистика по surgery? не изменилась, пропусков нет
remove_horse_data['outcome'].fillna(remove_horse_data['outcome'].mode()[0],inplace=True) # замена пропусков в outcome - модой
remove_horse_data['pain'].fillna(remove_horse_data['pain'].mode()[0],inplace=True)
remove_horse_data['temp_of_extremities'].fillna(remove_horse_data['temp_of_extremities'].mode()[0],inplace=True)
remove_horse_data.describe().round()


/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,surgery?,Age,rectal_temp,pulse,respiratory_rate,temp_of_extremities,pain,outcome
count,275.0,275.0,219.0,253.0,221.0,275.0,275.0,275.0
mean,1.0,1.0,38.0,67.0,28.0,2.0,3.0,2.0
std,0.0,0.0,1.0,24.0,15.0,1.0,1.0,1.0
min,1.0,1.0,35.0,30.0,8.0,1.0,1.0,1.0
25%,1.0,1.0,38.0,48.0,18.0,1.0,2.0,1.0
50%,1.0,1.0,38.0,60.0,24.0,3.0,3.0,1.0
75%,2.0,1.0,38.0,84.0,36.0,3.0,4.0,2.0
max,2.0,1.0,41.0,140.0,96.0,4.0,5.0,3.0


***Проверяем медианное значение  ректальной температуры, группируя по 'surgery?','pain'***

In [ ]:

print(remove_horse_data.groupby(['surgery?','pain'])['rectal_temp'].median())


surgery?  pain
1.0       1.0     38.10
          2.0     38.30
          3.0     38.15
          4.0     38.05
          5.0     38.00
2.0       1.0     38.30
          2.0     38.30
          3.0     38.00
          4.0     37.90
          5.0     38.60
Name: rectal_temp, dtype: float64


***Заменяем пропущенные значения в rectal_temp медианными значениями соответственно столбцам surgery? и pain***

In [ ]:
remove_horse_data['rectal_temp'].fillna(remove_horse_data.groupby(['surgery?','pain'])['rectal_temp'].transform('median'),inplace=True)

***Проверяем медианное значение  пульса и частоты дыхания, группируя по 'surgery?','pain'***

In [ ]:
print(remove_horse_data.groupby(['surgery?','pain'])['pulse'].median())
print(remove_horse_data.groupby(['surgery?','pain'])['respiratory_rate'].median())

surgery?  pain
1.0       1.0     48.0
          2.0     80.0
          3.0     60.0
          4.0     70.0
          5.0     80.0
2.0       1.0     48.0
          2.0     60.0
          3.0     52.0
          4.0     83.0
          5.0     96.0
Name: pulse, dtype: float64
surgery?  pain
1.0       1.0     16.0
          2.0     36.0
          3.0     24.0
          4.0     30.0
          5.0     30.0
2.0       1.0     20.0
          2.0     20.0
          3.0     25.0
          4.0     24.5
          5.0     28.0
Name: respiratory_rate, dtype: float64


***Заменяем пропущенные значения в pulse и respiratory_rate медианными значениями соответственно столбцам surgery? и pain***

In [ ]:
remove_horse_data['pulse'].fillna(remove_horse_data.groupby(['surgery?','pain'])['pulse'].transform('median'),inplace=True)
remove_horse_data['respiratory_rate'].fillna(remove_horse_data.groupby(['surgery?','pain'])['respiratory_rate'].transform('median'),inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
remove_horse_data.info() # пропусков нет
remove_horse_data.describe().round()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 275 entries, 0 to 298
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   surgery?             275 non-null    float64
 1   Age                  275 non-null    int64  
 2   rectal_temp          275 non-null    float64
 3   pulse                275 non-null    float64
 4   respiratory_rate     275 non-null    float64
 5   temp_of_extremities  275 non-null    float64
 6   pain                 275 non-null    float64
 7   outcome              275 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 19.3 KB


,surgery?,Age,rectal_temp,pulse,respiratory_rate,temp_of_extremities,pain,outcome
count,275.0,275.0,275.0,275.0,275.0,275.0,275.0,275.0
mean,1.0,1.0,38.0,67.0,28.0,2.0,3.0,2.0
std,0.0,0.0,1.0,23.0,14.0,1.0,1.0,1.0
min,1.0,1.0,35.0,30.0,8.0,1.0,1.0,1.0
25%,1.0,1.0,38.0,48.0,20.0,1.0,2.0,1.0
50%,1.0,1.0,38.0,60.0,24.0,3.0,3.0,1.0
75%,2.0,1.0,38.0,84.0,32.0,3.0,4.0,2.0
max,2.0,1.0,41.0,140.0,96.0,4.0,5.0,3.0


In [ ]:
remove_horse_data.head(10)


,surgery?,Age,rectal_temp,pulse,respiratory_rate,temp_of_extremities,pain,outcome
0,1.0,1,39.20,88.0,20.0,3.0,3.0,3.0
1,2.0,1,38.30,40.0,24.0,1.0,3.0,1.0
3,2.0,1,37.30,104.0,35.0,3.0,3.0,2.0
4,2.0,1,38.30,60.0,20.0,2.0,2.0,1.0
5,1.0,1,37.90,48.0,16.0,1.0,3.0,1.0
6,1.0,1,38.15,60.0,24.0,3.0,3.0,2.0
7,2.0,1,37.90,80.0,36.0,3.0,4.0,3.0
9,1.0,1,38.10,66.0,12.0,3.0,3.0,1.0
10,2.0,1,39.10,72.0,52.0,2.0,2.0,1.0
11,1.0,1,37.20,42.0,12.0,2.0,3.0,1.0
